In [1]:
# Valid SOURCES
s1 = 'gs://rl-gcloud-alpha/'
s2 = 'gs://rl-gcloud-alpha/output'
s3 = 'gs://rl-gcloud-alpha/test.txt'
s4 = 'gs://rl-gcloud-alpha/*'
s5 = 'gs://rl-gcloud-alpha/**'
s6 = 'gs://workshop-datasets/criteo-parque/'
s7 = 'gs://workshop-datasets/criteo-parque/*'

# Invalid GCS SOURCES
is1 = 'gs://rl-gcloud-alph/'
is2 = '/rl-gcloud-alpha/output'
is3 = 'gs://rl-gcloud-alpha/test' # file or directory

# Valid Local SOURCES
l1 = '/home/renatoleite/data'
l2 = '/home/renatoleite/output'

In [2]:
from gcs_copy_utils import GcsCopyUtils

In [3]:
test = GcsCopyUtils(s1, l2, l2, 'gs://rl-gcloud-alpha/results', True, 'txt')

In [4]:
print(test.compose_gcloud_download_cmd())
print(test.compose_gcloud_upload_cmd())

['gcloud', 'alpha', 'storage', 'cp', 'gs://rl-gcloud-alpha/**.txt', '/home/renatoleite/output']
['gcloud', 'alpha', 'storage', 'cp', '-r', '/home/renatoleite/output', 'gs://rl-gcloud-alpha/results']


In [5]:
download = test.compose_gcloud_download_cmd()
upload = test.compose_gcloud_upload_cmd()

In [6]:
test.execute_gcloud_cmd(download)

{'returncode': 0,
 'stdout': '',
 'stderr': 'Copying gs://rl-gcloud-alpha/output/opa.txt to file:///home/renatoleite/output/opa.txt\nCopying gs://rl-gcloud-alpha/output/s1/a.txt to file:///home/renatoleite/output/a.txt\n  \nCopying gs://rl-gcloud-alpha/output/s2/b.txt to file:///home/renatoleite/output/b.txt\nCopying gs://rl-gcloud-alpha/results/a.txt to file:///home/renatoleite/output/a.txt\nSkipping FileDownloadTask for file:///home/renatoleite/output/a.txt. This can occur if a cp command results in multiple writes to the same resource.\nCopying gs://rl-gcloud-alpha/results/b.txt to file:///home/renatoleite/output/b.txt\nSkipping FileDownloadTask for file:///home/renatoleite/output/b.txt. This can occur if a cp command results in multiple writes to the same resource.\nCopying gs://rl-gcloud-alpha/results/opa.txt to file:///home/renatoleite/output/opa.txt\nSkipping FileDownloadTask for file:///home/renatoleite/output/opa.txt. This can occur if a cp command results in multiple writes t

In [7]:
test.execute_gcloud_cmd(upload)

{'returncode': 0,
 'stdout': '',
 'stderr': 'Copying file:///home/renatoleite/output/test.txt to gs://rl-gcloud-alpha/results/test.txt\nCopying file:///home/renatoleite/output/a.txt to gs://rl-gcloud-alpha/results/a.txt\n  \nCopying file:///home/renatoleite/output/opa.txt to gs://rl-gcloud-alpha/results/opa.txt\nCopying file:///home/renatoleite/output/renato.txt to gs://rl-gcloud-alpha/results/renato.txt\nCopying file:///home/renatoleite/output/b.txt to gs://rl-gcloud-alpha/results/b.txt\n.\n\nAverage throughput: 645.0B/s\n'}